In [3]:
import pandas as pd

In [6]:
demand = pd.read_clipboard()
demand

,Demand,D_qty
0,East Coast BB,4080
1,East Coast SB,1121
2,West Coast BB,624
3,Salinas BB,5585
4,EU,4607
5,EEMEA,0
6,LA,1703
7,AP - China,3571
8,China,5345


In [46]:
supply = pd.read_clipboard()
supply

,Supplier,Location,Max_Capacity
0,BC,BC San Pedro,12480
1,BC,BC Malaysia,7200
2,BC,BC Pasir Gudang,7200
3,Olam,Olam Singapore,7500
4,Olam,Olam Holland,6000


In [44]:
base_cost = pd.read_clipboard()
base_cost

,Location,Cost
0,BC San Pedro,3496
1,BC Malaysia,3942
2,BC Pasir Gudang,3942
3,Olam Singapore,3836
4,Olam Holland,3278


In [80]:
df = pd.read_clipboard()
df3=df.rename_axis('dest').reset_index()
trans_cost=df3.melt(id_vars=["dest"], var_name="orig", value_name="Value")
trans_cost

,dest,orig,Value
0,East Coast BB,BC San Pedro,111
1,East Coast SB,BC San Pedro,111
2,West Coast BB,BC San Pedro,511
3,Salinas BB,BC San Pedro,620
4,EU,BC San Pedro,75
5,EEMEA,BC San Pedro,163
6,LA,BC San Pedro,120
7,AP - China,BC San Pedro,295
8,China,BC San Pedro,345
9,East Coast BB,BC Malaysia,240


In [120]:
import pandas as pd
import numpy as np
from pulp import *
from collections import defaultdict

In [121]:
#suppliers = range(supply.shape[0])
#customers = range(demand.shape[0])

suppliers = supply['Location'].unique()
customers = demand['Demand'].unique()

BC_supp=['BC San Pedro', 'BC Malaysia', 'BC Pasir Gudang']

flow = [(s,c) for s in suppliers for c in customers]

prob = LpProblem("Least_cost_Network",LpMinimize)

flows = LpVariable.dicts("s-D-flow",flow, lowBound = 0, cat="Continuous")
supp = LpVariable.dicts("Supp_select",suppliers, cat="Binary") #in million-tons
penalty = LpVariable('Is_penalty', cat="Binary")


In [122]:
dict_demand=demand.set_index(['Demand']).D_qty.to_dict()
dict_supply=supply.set_index(['Location']).Max_Capacity.to_dict()

dict_base_cost=base_cost.set_index(['Location']).Cost.to_dict()
dict_trans_cost=trans_cost.set_index(['orig','dest']).Value.to_dict()

In [123]:
# objective function

## Delivered cost = Base Cost + Freight Cost + Any taxes/duties

prob += (
    lpSum([supp[s] * dict_base_cost[s]  for s in suppliers]) + lpSum([flows[f] * dict_trans_cost[f]  for f in flow]) + 125*penalty
)


In [124]:
## Demand Constraint

for c in customers:
    prob += (
        lpSum( flows[(s,c)] for s in suppliers ) >= dict_demand[c]
    )
    

In [125]:
## Supply Constraint
## Flow cannot be there if supply point is not active.

for s in suppliers:
    prob += (
        lpSum( flows[(s,c)] for c in customers ) <= dict_supply[s]*supp[s]
    )


In [126]:
## Penalty incurred If BC Supply <19000 tons.

prob += (
        lpSum( flows[(s,c)] for s in suppliers for c in customers if s in BC_supp) <= 26880*(1-penalty) + 19000*(penalty)
    )


In [128]:
prob.writeLP("pricing.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Z003CNF/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/my/1126m1255k36stt7f2r98h_w0000gq/T/5fd9f3417d0f4738bb60027f34a080f9-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/my/1126m1255k36stt7f2r98h_w0000gq/T/5fd9f3417d0f4738bb60027f34a080f9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 207 RHS
At line 223 BOUNDS
At line 230 ENDATA
Problem MODEL has 15 rows, 51 columns and 123 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.41764e+06 - 0.00 seconds
Cgl0004I processed model has 14 rows, 45 columns (5 integer (5 of which binary)) and 109 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.676325
Cbc0038I Pass   1: suminf.    0.09111 (1) obj. 3.80927e+06 iterations 3
Cbc0038I Solu

In [130]:
output = []
for v in prob.variables():
    if v.varValue > 0.01 :
        output.append(str(v.name) + "=" + str(v.varValue))
        
df = pd.DataFrame({'col': output})
df

,col
0,Supp_select_BC_Malaysia=1.0
1,Supp_select_BC_San_Pedro=1.0
2,Supp_select_Olam_Holland=1.0
3,Supp_select_Olam_Singapore=1.0
4,"s_D_flow_('BC_Malaysia',_'AP___China')=3571.0"
5,"s_D_flow_('BC_Malaysia',_'LA')=1703.0"
6,"s_D_flow_('BC_San_Pedro',_'EU')=4607.0"
7,"s_D_flow_('BC_San_Pedro',_'East_Coast_BB')=4080.0"
8,"s_D_flow_('BC_San_Pedro',_'East_Coast_SB')=1121.0"
9,"s_D_flow_('Olam_Holland',_'Salinas_BB')=5376.0"


In [127]:
print(prob)

Least_cost_Network:
MINIMIZE
125*Is_penalty + 3942*Supp_select_BC_Malaysia + 3942*Supp_select_BC_Pasir_Gudang + 3496*Supp_select_BC_San_Pedro + 3278*Supp_select_Olam_Holland + 3836*Supp_select_Olam_Singapore + 60*s_D_flow_('BC_Malaysia',_'AP___China') + 60*s_D_flow_('BC_Malaysia',_'China') + 41*s_D_flow_('BC_Malaysia',_'EEMEA') + 162*s_D_flow_('BC_Malaysia',_'EU') + 240*s_D_flow_('BC_Malaysia',_'East_Coast_BB') + 240*s_D_flow_('BC_Malaysia',_'East_Coast_SB') + 110*s_D_flow_('BC_Malaysia',_'LA') + 445*s_D_flow_('BC_Malaysia',_'Salinas_BB') + 240*s_D_flow_('BC_Malaysia',_'West_Coast_BB') + 60*s_D_flow_('BC_Pasir_Gudang',_'AP___China') + 60*s_D_flow_('BC_Pasir_Gudang',_'China') + 41*s_D_flow_('BC_Pasir_Gudang',_'EEMEA') + 162*s_D_flow_('BC_Pasir_Gudang',_'EU') + 240*s_D_flow_('BC_Pasir_Gudang',_'East_Coast_BB') + 240*s_D_flow_('BC_Pasir_Gudang',_'East_Coast_SB') + 110*s_D_flow_('BC_Pasir_Gudang',_'LA') + 445*s_D_flow_('BC_Pasir_Gudang',_'Salinas_BB') + 240*s_D_flow_('BC_Pasir_Gudang',_'We